**Ensure all other Jupyter kernels are stopped when running GPU-accelerated labs.**

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import boston_housing
from tensorflow.keras import models
from tensorflow.keras import layers

np.random.seed(1671) # Reproducibility, same seed used in lab sheet.

print("TensorFlow version:", tf.__version__)
print("Detected devices:", tf.config.list_physical_devices())
# assert tf.test.is_built_with_cuda() 
# ? Throws an error if CUDA isn't working properly (GPU disconnected).

TensorFlow version: 2.11.1
Detected devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [2]:
(trainData, trainTargets), (testData, testTargets) = boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


In [3]:
mean = trainData.mean(axis=0)
trainData -= mean
std = trainData.std(axis=0)
trainData /= std
testData -= mean
testData /= std

In [4]:
def buildModel():
    model = models.Sequential()
    
    model.add(layers.Dense(64, activation = "relu", 
                           input_shape = (trainData.shape[1],)))
    model.add(layers.Dense(64, activation = "relu"))
    model.add(layers.Dense(1))
    
    model.compile(optimizer = "rmsprop", loss = "mse", metrics = ["mae"])
    return model

In [ ]:
# Will be appended to in order to hold all model metrics.
allScores = []

def kFold(k, numEpochs, trainData = trainData):
    numValSamples = len(trainData) // k
    
    for i in range(k):
        print("Processing fold ", i + 1)
        valData = trainData[i * numValSamples: (i + 1) * numValSamples] 
        valTargets = trainTargets[i * numValSamples: (i + 1) * numValSamples]
        partialTrainData = np.concatenate(
            [trainData[: i * numValSamples], 
             trainData[(i + 1) * numValSamples:]],
            axis = 0
        ) 
        partialTrainTargets = np.concatenate(
            [trainTargets[: i * numValSamples], 
             trainTargets[(i + 1) * numValSamples:]],
            axis = 0
        ) 
        
        model = buildModel()
        
        model.fit(partialTrainData, partialTrainTargets, epochs = numEpochs,
                  batch_size = 1, verbose = 1)
        
        valMSE, valMAE = model.evaluate(valData, valTargets)
        allScores.append(valMAE)

In [6]:
kFold(k = 4, numEpochs = 100)

Processing fold  0
Epoch 1/100
303/303 [==============================] - 0s 571us/step - loss: 229.9498 - mae: 11.6185
Epoch 2/100
303/303 [==============================] - 0s 561us/step - loss: 32.5936 - mae: 3.8861
Epoch 3/100
303/303 [==============================] - 0s 588us/step - loss: 23.6775 - mae: 3.1834
Epoch 4/100
303/303 [==============================] - 0s 560us/step - loss: 19.4957 - mae: 2.8940
Epoch 5/100
303/303 [==============================] - 0s 560us/step - loss: 18.3771 - mae: 2.8212
Epoch 6/100
303/303 [==============================] - 0s 560us/step - loss: 16.5934 - mae: 2.6113
Epoch 7/100
303/303 [==============================] - 0s 540us/step - loss: 15.4917 - mae: 2.5508
Epoch 8/100
303/303 [==============================] - 0s 530us/step - loss: 14.9324 - mae: 2.4950
Epoch 9/100
303/303 [==============================] - 0s 545us/step - loss: 14.1657 - mae: 2.4242
Epoch 10/100
303/303 [==============================] - 0s 543us/step - loss: 13.3665 - 

In [7]:
for i, score in enumerate(allScores):
    print(f"Fold {i} MAE: {score}")
    
print(f"Overall MAE: {np.mean(allScores)}")

Fold 0 MAE: 2.5176401138305664
Fold 1 MAE: 2.9134552478790283
Fold 2 MAE: 2.446413278579712
Fold 3 MAE: 2.4999310970306396
Overall MAE: 2.5943599343299866
